In [1]:
!pip install causalnex

  Using cached causalnex-0.12.1-py3-none-any.whl (153 kB)
  Using cached pgmpy-0.1.19-py3-none-any.whl (1.9 MB)
  Using cached scipy-1.13.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
  Using cached numpy-1.23.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached pyvis-0.3.2-py3-none-any.whl (756 kB)
  Using cached scikit_learn-1.5.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
  Using cached pandas-1.5.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached ipython-8.18.1-py3-none-any.whl (808 kB)
  Using cached pathos-0.3.2-py3-none-any.whl (82 kB)
  Using cached stack_data-0.6.3-py3-none-any.whl (24 kB)
  Using cached multiprocess-0.70.16-py39-none-any.whl (133 kB)
  Using cached pox-0.3.4-py3-none-any.whl (29 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached ppft-1.7.6.8-py3-none-any.whl (56 kB)
  Using cached statsmodels-0.14.2-cp39-cp39-manylinux_2_17_x86_64

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import os
# os.chdir("../../")

# from src import display_df

In [2]:
df = pd.read_csv("features_data.csv")

In [3]:
df.head()

,order_id,trip_origin,trip_destination,trip_start_time,trip_end_time,id,driver_id,driver_action,drivers_lat,drivers_lon,...,day_of_month,month,trip_start_date,trip_end_date,trip_origin_latitude,trip_origin_longitude,trip_destination_latitude,trip_destination_longitude,is_weekend,is_holiday
0,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,1.0,243828.0,accepted,6.602207,3.270465,...,1,July,2021-07-01,2021-07-01,6.601042,3.276634,6.450107,3.391615,0,0
1,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,2.0,243588.0,rejected,6.592097,3.287445,...,1,July,2021-07-01,2021-07-01,6.601042,3.276634,6.450107,3.391615,0,0
2,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,3.0,243830.0,rejected,6.596133,3.281784,...,1,July,2021-07-01,2021-07-01,6.601042,3.276634,6.450107,3.391615,0,0
3,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,4.0,243539.0,rejected,6.596142,3.280526,...,1,July,2021-07-01,2021-07-01,6.601042,3.276634,6.450107,3.391615,0,0
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,5.0,171653.0,rejected,6.609232,3.288800,...,1,July,2021-07-01,2021-07-01,6.601042,3.276634,6.450107,3.391615,0,0


#### Drop irrelevant columns

In [4]:
# Drop columns not needed for training
drop_col = ['order_id', 
            'trip_start_time', 
            'trip_end_time',
            "trip_origin",
            "trip_destination",
            "id", 
            "driver_id", 
            "drivers_lat", 
            "drivers_lon", 
            "trip_start_date", 
            "trip_end_date",
            "drivers_lat",
            "drivers_lon",
            "trip_start_date",
            "trip_end_date",
            "trip_origin_latitude",
            "trip_origin_longitude",
            "trip_destination_latitude",
            "trip_destination_longitude",
            "is_holiday",
            "day_of_week",
            "day_of_month",
            "month"
          ]

In [5]:
data = df.drop(columns=drop_col)
data.head()
# display_df(data)

,driver_action,hour_of_day,is_weekend
0,accepted,9,0
1,rejected,9,0
2,rejected,9,0
3,rejected,9,0
4,rejected,9,0


#### Make the data numeric. We can use label encoding for the categorical columns.

###### Display the non numeric columns

In [6]:
import numpy as np

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

['driver_action']


In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

# display_df(struct_data)

### Apply NOTEARS algorithm to learn the structure of the data

In [8]:

from causalnex.structure.notears import from_pandas

# struct_data = struct_data.head(1000)
sm = from_pandas(struct_data)


#### Visualize the learned StructureModel model using the plot function

In [9]:
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("fully_connected.html")

fully_connected.html


### Remove weak Edges

In [19]:

sm.remove_edges_below_threshold(0.8)
viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)
viz.show("thresholded.html")

thresholded.html


In [27]:
# Add the edge from 'hour_of_day' to 'driver_action' 
sm.add_edge('hour_of_day', 'driver_action')
sm.add_edge('is_weekend', 'driver_action')

In [22]:
# # Learn structure (with optional constraints)
sm = from_pandas(struct_data,tabu_edges=[("is_weekend", "hour_of_day"), ("driver_action", "hour_of_day")])


In [30]:
# sm.remove_edges_below_threshold(0.8)

viz = plot_structure(
    sm,
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK,
)

viz.toggle_physics(False)
viz.show("late_modified.html")

late_modified.html


In [20]:
data.head()

,driver_action,hour_of_day,is_weekend
0,accepted,9,0
1,rejected,9,0
2,rejected,9,0
3,rejected,9,0
4,rejected,9,0


## Cardinality of Categorical Features

To reduce the cardinality of categorical features we can define a map {old_value: new_value}, and use this to update the feature. 

In [31]:
# Create a copy of the DataFrame
discretised_data = data.copy()

# Create a dictionary to store unique values for each column
data_vals = {col: data[col].unique() for col in data.columns}

# Create a dictionary to map driver_action values to 'accepted' and 'rejected'
driver_action_map = {v: v for v in data_vals['driver_action']}

# Create a dictionary to map hour_of_day values to 'morning' or 'afternoon/night'
hour_of_day_map = {v: 'morning' if v <= 12 else 'afternoon/night' for v in data_vals['hour_of_day']}

# Create a dictionary to map is_weekend values to weekend or not_weekend
is_weekend_map = {v: 'weekday' if v == 0 else 'weekend' for v in data_vals['is_weekend']}

# Map the values in the driver_action and hour_of_day columns
discretised_data["driver_action"] = discretised_data["driver_action"].map(driver_action_map)
discretised_data["hour_of_day"] = discretised_data["hour_of_day"].map(hour_of_day_map)
discretised_data['is_weekend'] = discretised_data['is_weekend'].map(is_weekend_map)

# Print the first 5 rows of the columns driver_action and hour_of_day
print(discretised_data[['driver_action', 'hour_of_day', 'is_weekend']].head(20).to_markdown(index=False, numalign="left", stralign="left"))


| driver_action   | hour_of_day   | is_weekend   |
|:----------------|:--------------|:-------------|
| accepted        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| accepted        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning       | weekday      |
| rejected        | morning    

In [32]:
discretised_data.head()

,driver_action,hour_of_day,is_weekend
0,accepted,morning,weekday
1,rejected,morning,weekday
2,rejected,morning,weekday
3,rejected,morning,weekday
4,rejected,morning,weekday


#### Split the data into train and test sets

In [39]:
# Split 90% train and 10% test
from sklearn.model_selection import train_test_split

train, test = train_test_split(discretised_data, train_size=0.9, test_size=0.1, random_state=7)

### Fit a bayesian network

In [35]:
from causalnex.network import BayesianNetwork
from causalnex.structure.structuremodel import StructureModel

# Having my learned model 'sm' from a previous step

# Convert the StructureModel (sm) into a BayesianNetwork
bn = BayesianNetwork(sm)

#### With the learnt structure model from earlier and the discretised data, we can now fit the probability distrbution of the Bayesian Network.


In [37]:
bn = bn.fit_node_states(discretised_data)  # Fit node states first

#### Fit Conditional Probability Distributions


In [40]:
bn = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")

##### Once we have the CPDs, we can inspect them through the cpds property, which is a dictionary of node->cpd.



In [41]:
bn.cpds["driver_action"]

hour_of_day   afternoon/night             morning          
is_weekend            weekday   weekend   weekday   weekend
driver_action                                              
accepted             0.015104  0.016578  0.018374  0.022013
rejected             0.984896  0.983422  0.981626  0.977987

#### Classification Report

In [48]:
from causalnex.evaluation import classification_report

classification_report(bn, test, "driver_action")

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

{'driver_action_accepted': {'precision': 0.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 2574.0},
 'driver_action_rejected': {'precision': 0.9834760614736734,
  'recall': 1.0,
  'f1-score': 0.9916692019393217,
  'support': 153200.0},
 'accuracy': 0.9834760614736734,
 'macro avg': {'precision': 0.4917380307368367,
  'recall': 0.5,
  'f1-score': 0.49583460096966087,
  'support': 155774.0},
 'weighted avg': {'precision': 0.9672251634917686,
  'recall': 0.9834760614736734,
  'f1-score': 0.975282921008025,
  'support': 155774.0}}

### Machine learning

In [94]:
# 2. Define selected features (if not already done):
selected_features = ['is_weekend_weekend', 'hour_of_day_morning']

In [95]:
# 3. Preprocess Training Data ONLY:
train_encoded = pd.get_dummies(train.copy(), columns=selected_features)
le = LabelEncoder()
train_encoded["driver_action"] = le.fit_transform(train_encoded["driver_action"])


In [96]:
# 4. Preprocess Testing Data (Using Training Data's Transformations):
test_encoded = pd.get_dummies(test.copy(), columns=selected_features)


In [97]:
# Align columns to match training data
missing_cols = set(train_encoded.columns) - set(test_encoded.columns)
for col in missing_cols:
    test_encoded[col] = 0


In [98]:
test_encoded = test_encoded[train_encoded.columns]  # Ensure same column order

test_encoded["driver_action"] = le.transform(test_encoded["driver_action"])


In [99]:
# 5. Extract Features and Target Variables
X_train = train_encoded.drop("driver_action", axis=1)   # All features for training
y_train = train_encoded["driver_action"]                # Target variable for training
X_test = test_encoded.drop("driver_action", axis=1)     # All features for testing
y_test = test_encoded["driver_action"]                  # Target variable for testing

In [100]:
# Define selected features from your Bayesian network analysis
# selected_features = ['is_weekend_weekday', 'is_weekend_weekend', 'hour_of_day_afternoon/night', 'hour_of_day_morning'] 

X_train_selected = train[selected_features]          # Selected features for training
X_test_selected = test[selected_features]           # Selected features for testing


## ---- Random Forest ----

In [101]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [102]:
# All Features
rf_all = RandomForestClassifier(random_state=7)
rf_all.fit(X_train, y_train)
y_pred = rf_all.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest (All Features): Accuracy = {accuracy:.8f}")

Random Forest (All Features): Accuracy = 0.983476


In [103]:
# Selected Features
rf_selected = RandomForestClassifier(random_state=7)
rf_selected.fit(X_train_selected, y_train)
y_pred = rf_selected.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest (Selected Features): Accuracy = {accuracy:.8f}")


Random Forest (Selected Features): Accuracy = 0.9835


## --- XGBoost ---

In [104]:
# --- XGBoost ---
from xgboost import XGBClassifier

# All Features
xgb_all = XGBClassifier(random_state=7)
xgb_all.fit(X_train, y_train)
y_pred = xgb_all.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost (All Features): Accuracy = {accuracy:.9f}")

XGBoost (All Features): Accuracy = 0.983476061


In [105]:
# Selected Features
xgb_selected = XGBClassifier(random_state=7)
xgb_selected.fit(X_train_selected, y_train)
y_pred = xgb_selected.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost (Selected Features): Accuracy = {accuracy:.9f}")

XGBoost (Selected Features): Accuracy = 0.983476061


## --- Gradient Boosting ---


In [106]:

# All Features
gb_all = GradientBoostingClassifier(random_state=7)
gb_all.fit(X_train, y_train)
y_pred = gb_all.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Gradient Boosting (All Features): Accuracy = {accuracy:.9f}")

Gradient Boosting (All Features): Accuracy = 0.983476061


In [107]:
# Selected Features
gb_selected = GradientBoostingClassifier(random_state=7)
gb_selected.fit(X_train_selected, y_train)
y_pred = gb_selected.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Gradient Boosting (Selected Features): Accuracy = {accuracy:.9f}")


Gradient Boosting (Selected Features): Accuracy = 0.983476061


## --- Logistic Regression ---

In [88]:
# All Features
lr_all = LogisticRegression(random_state=7)
lr_all.fit(X_train, y_train)
y_pred = lr_all.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression (All Features): Accuracy = {accuracy:.9f}")

Logistic Regression (All Features): Accuracy = 0.983476061


In [87]:
# Selected Features
lr_selected = LogisticRegression(random_state=7)
lr_selected.fit(X_train_selected, y_train)
y_pred = lr_selected.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression (Selected Features): Accuracy = {accuracy:.9f}")

Logistic Regression (Selected Features): Accuracy = 0.983476061
